# Create a Slice and Run iPerf3



## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  


In [ ]:
slice_name = 'iPerf3-2'
site1 = fablib.get_random_site()
print(f"Sites: {site1}")
site1='LBNL'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

net1 = slice.add_l2network(name=network_name, subnet=subnet)


hosts = [f"{site1.lower()}-w1", f"{site1.lower()}-w2", f"{site1.lower()}-w3"]

for h in hosts:

    # Node1
    node1 = slice.add_node(name=f"{h}-a", cores=16, ram=32, site=site1, image='docker_rocky_8', host=f"{h}.fabric-testbed.net")
    #node1.add_fabnet()

    iface1 = node1.add_component(model=model_name, name=nic_name).get_interfaces()[0]
    #iface1.set_mode('auto')
    net1.add_interface(iface1)

    node1.add_post_boot_upload_directory('node_tools','.')
    node1.add_post_boot_execute('sudo node_tools/host_tune.sh')
    node1.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
    node1.add_post_boot_execute('docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ')

#Submit Slice Request
slice.submit();

### CPU Pinning and Numatune Setting

In [ ]:
for node in slice.get_nodes():
    # Find CPU Allocation on the Host
    cpu_info = node.perform_operational_action(operation="cpuinfo")
    pinned_cpus = cpu_info.get(node.get_host()).get('pinned_cpus')
    
    # Find Numa Node for the NIC
    numa_node = node.get_component(name=nic_name).get_numa_node()
    
    # Find CPUs assigned to the numa node
    ranges = cpu_info.get(node.get_host()).get(f"NUMA node{numa_node} CPU(s):")

    numa_cpu_range = []
    for r in ranges.split(","):
        start, end = map(int, r.split("-"))
        numa_cpu_range.extend(map(str, range(start, end + 1)))

    available_cpus = list(set(numa_cpu_range) - set(pinned_cpus))
    idx = node.get_cores()
    vcpu_cpu_map = []
    for x in available_cpus:
        temp = {'vcpu': str(node.get_cores() - idx), 'cpu': x}
        vcpu_cpu_map.append(temp)
        idx -= 1

    print(vcpu_cpu_map)
    
    status = node.perform_operational_action(operation="cpupin", vcpu_cpu_map=vcpu_cpu_map)
    print(f"CPU Pinning complete for node: {node.get_name()}")
          
    status = node.perform_operational_action(operation="numatune", node_set=[str(numa_node)])
    print(f"Numa tuning complete for node: {node.get_name()}")
    
    status = node.perform_operational_action(operation="reboot")
    print(f"Node: {node.get_name()} rebooted")

## Configure IP addresses on the VM and download fabric tools

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
    node_iface = node.get_interface(network_name=network_name) 
    node_addr = available_ips.pop(0)
    node_iface.ip_addr_add(addr=node_addr, subnet=subnet)

    stdout, stderr = node.execute(f'ip addr show {node_iface.get_device_name()}')

    stdout1, stderr1 = node.execute("sudo systemctl enable docker")
    stdout1, stderr1 = node.execute("sudo systemctl start docker")    
    stdout1, stderr1 = node.execute("sudo docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ")
    stdout1, stderr1 = node.execute("sudo systemctl enable docker")
    stdout1, stderr1 = node.execute("sudo systemctl start docker")    

In [ ]:
slice = fablib.get_slice(slice_name)
slice.list_interfaces();

## Run iperf3

In [ ]:
slice = fablib.get_slice(slice_name)


source_name = f"{site1.lower()}-w1-a"
source_node = slice.get_node(name=source_name)

source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()

node_names = [f"{site1.lower()}-w1-a", f"{site1.lower()}-w2-a", f"{site1.lower()}-w3-a"]

for i in range(len(node_names)):
    source_node = slice.get_node(name=node_names[i])
    source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()
    
    for j in range(len(node_names)):
        if i==j:
            continue
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        stdout1, stderr1 = source_node.execute("docker run -d --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            "iperf3 -s -1"
                                            , quiet=True, output_file=f"{source_node.get_name()}.log");

        print(f"Source:  {node_names[i]} to Dest: {node_names[j]}")
        dest_node = slice.get_node(name=node_names[j])

        stdout2, stderr2 = dest_node.execute("docker run --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            f"iperf3 -c {source_addr} -P 10 -t 30 -i 10 -O 10"
                                            , quiet=False, output_file=f"{dest_node.get_name()}.log");
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()